In [1]:
%load_ext autoreload
%autoreload 2


%matplotlib inline
%cd C:\MAD4AG

from tqdm.auto import tqdm
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import folium


warnings.filterwarnings('ignore')

C:\MAD4AG


In [3]:
file = r'.\dbs\twins\03-19-schedule-0.pkl'

df_act_schedule_0 = pd.read_pickle(file)



In [44]:
individual_uid= '259ad3e2-3e4e-4b56-b4b3-7bb87e634648-1'




individual = df_act_schedule_0[(df_act_schedule_0['uid']==individual_uid)]

individual_trip = df_act_schedule_0[(df_act_schedule_0['uid']==individual_uid)]
individual_trip['lat_dest'] =individual_trip['lat'].shift(1)
individual_trip['lng_dest'] =individual_trip['lng'].shift(1)

In [48]:
map = folium.Map(individual[['lat','lng']][individual.purpose=='Home'].iloc[0], zoom_start=12)#, crs='EPSG4326')



for point in range(1, len(individual_trip)):
    folium.PolyLine([(individual_trip.iloc[point]['lat'], individual_trip.iloc[point]['lng']), (individual_trip.iloc[point]['lat_dest'], individual_trip.iloc[point]['lng_dest'])]).add_to(map)

for point in range(0, len(individual)):
    folium.Circle([individual.iloc[point]['lat'], individual.iloc[point]['lng']], fill_color="orange", fill_opacity=0.).add_to(map)

folium.Marker(individual[['lat','lng']][individual.purpose=='Home'].iloc[0], icon=folium.Icon( icon='home', prefix='fa')).add_to(map)

map